In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import * 

In [ ]:
def gen_data(params,nobs=100,var=5):
    wtp,cost,beta1,beta2,emc1,emc2=params
    emc = emc1,emc2
    passive_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2']) 
    active_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])
    seq_result = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'])

    for i in range(0,nobs):
        #sequential
        mc1 = np.random.normal(loc=0, scale=var, size=(2,)) + np.array([emc1,emc2])
        mc2 = np.random.normal(loc=0, scale=1, size=(2,))
        mc = mc1+mc2

        #seq results
        phi1,phi2 = seq_bargain(25,cost,wtp,mc,betas=[beta1,beta2])
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        seq_result = pd.concat( [seq_result, row] )


        #active results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=True,betas=[beta1,beta2])
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        active_result = pd.concat( [active_result, row] )


        #passive results
        phi1,phi2 =simult_bargain(25,25, cost,wtp,mc, active=False)
        phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,mc)
        row = pd.DataFrame(columns=['phi1','phi2','p1','p2','s1','s2','c1','c2'],data=[[phi1,phi2,p1,p2,s1,s2,mc1[0],mc1[1]]])
        passive_result = pd.concat( [passive_result, row] )
    return seq_result, active_result,passive_result


param_list = [[25,5,.5,.5,0,0],[20,5,.5,.5,0,0],[25,10,.5,.5,0,0],[25,5,.5,.1,0,0],[25,5,.1,.5,0,0],[25,5,.5,.5,3,3]]

for i in range(len(param_list)):
    seq_result,active_result,passive_result = gen_data(param_list[i],nobs=10000,var=5)
    seq_result.to_csv('fake_data/seq_data_%s.csv'%i)
    active_result.to_csv('fake_data/active_data_%s.csv'%i)
    passive_result.to_csv('fake_data/passive_data_%s.csv'%i)

In [ ]:
#calculate moment conditions for the linear model...
def solve_parameters_obj(data,params, active=False,seq=False):
    wtp,cost,beta1,beta2,mc1,mc2 = params
    
    data_moments = list(data.mean())
    phi1,phi2 = None,None
    if seq:
        phi1,phi2 = seq_bargain(25, cost, wtp, [mc1,mc2],betas=[beta1,beta2])
    elif active:
        phi1,phi2 = simult_bargain(25,25, cost, wtp, [mc1,mc2],active=True,betas=[beta1,beta2])
    else:
        phi1,phi2 = simult_bargain(25,25, cost, wtp, [mc1,mc2],active=False,betas=[beta1,beta2])
    
    phi1,phi2,p1,p2,s1,s2 = solve_eq(phi1,phi2,cost,wtp,[mc1,mc2])

    computed_moments = [phi1,phi2,p1,p2,s1,s2]
    
    obj = 0
    for i in range(6):
        obj = obj + (data_moments[i] - computed_moments[i])**2
    
    return obj

print(seq_result)

print( solve_parameters_obj(seq_result,[25,5,.5,.5,0,0], seq=True) )
print( solve_parameters_obj(seq_result,[25,5,.5,.5,0,0], active=True) )

In [ ]:
def solve_parameters(data,active=False,seq=False): 
    obj = lambda params: solve_parameters_obj(data,params,active=active,seq=seq)
    params = minimize(obj, [25,5,.5,.5,0,0], options={'fatol':1e-2,'xatol':1e-2,'maxiter':20},method='Nelder-Mead').x # 
    return params

print(solve_parameters(seq_result,active=False,seq=True))
print(solve_parameters(active_result,active=True,seq=False))
print(solve_parameters(passive_result,active=False,seq=False))